In [1]:
import pandas as pd
import geopandas as gpd 
import numpy as np
import fiona
from shapely.geometry import Polygon, LineString, Point

In [2]:
path  = 'G:\我的云端硬盘\PC5287_M.SC Coursework Thesis for Physics And Technology\data set'

## Planning Area geometry

In [3]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' 
pa_plg = pa = gpd.read_file(path+'\master-plan-2019-planning-area-boundary-no-sea\planning-boundary-area.kml', driver='KML')
pa_plg = pa_plg.to_crs(3414)

def extract_desc_pa(desc_pa):
    return desc_pa.split('PLN_AREA_N')[1].split('<td>')[1].split('<')[0]

pa_plg['PLN_AREA_N'] = pa_plg.Description.apply(extract_desc_pa)
pa_plg.head()

,Name,Description,geometry,PLN_AREA_N
0,kml_1,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((26228.902 30746.172 0.000, 26280.4...",BUKIT MERAH
1,kml_2,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((21448.724 41356.741 0.000, 21508.5...",BUKIT PANJANG
2,kml_3,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((24031.387 36694.742 0.000, 24076.6...",BUKIT TIMAH
3,kml_4,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((24934.967 44017.724 0.000, 24953.6...",CENTRAL WATER CATCHMENT
4,kml_5,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((45094.369 42203.459 0.000, 45174.6...",CHANGI


Define the function to get Planning area name for each stops/MRT stations, which will be used later

In [4]:
pa_plg_geo = pa_plg.set_index(['PLN_AREA_N'])['geometry'].to_dict()

def get_PLA_AREA_N(coordinate):
    for key in pa_plg_geo:
        if coordinate.within(pa_plg_geo.get(key)):
            return key 

## Bus flux

In [37]:
# read the bus transpotation volume file 
bus_vol = pd.read_csv(path + '\BusODvolume.2019.01.csv')
bus_vol.head()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,2019-01,WEEKDAY,6,BUS,95209,93019,34
1,2019-01,WEEKENDS/HOLIDAY,15,BUS,10149,50051,1
2,2019-01,WEEKDAY,15,BUS,10149,50051,2
3,2019-01,WEEKDAY,23,BUS,64319,63139,1
4,2019-01,WEEKDAY,8,BUS,85049,92149,1


In [38]:
# read the bus stops' location data 
bus_loc = pd.read_csv(path + '\Bus_stops_2019.02.01.csv')

# transform the dataframe into GeoDataFrame format, 
# and then transform the cordinate into SVY21 
bus_loc = gpd.GeoDataFrame(bus_loc, geometry=gpd.points_from_xy(bus_loc.Longitude,bus_loc.Latitude))
bus_loc = bus_loc.drop(columns=['Latitude', 'Longitude'])
bus_loc = bus_loc.set_crs(epsg=4326)
bus_loc = bus_loc.to_crs(3414)
bus_loc.head()

,RequestTimestamp,BusStopCode,RoadName,Description,geometry
0,2019.02.01.03.00.00,43759,Bt Batok Rd,Blk 443D,POINT (17269.854 37561.743)
1,2019.02.01.03.00.00,43761,Bt Batok Rd,Opp Dulwich Coll,POINT (17354.077 36989.704)
2,2019.02.01.03.00.00,43769,Bt Batok Rd,Dulwich Coll/Blk 445,POINT (17371.596 37069.306)
3,2019.02.01.03.00.00,43771,Bt Batok Rd,Bef Bt Batok West Ave 2,POINT (17766.452 38229.766)
4,2019.02.01.03.00.00,43779,Bt Batok Rd,Aft Bt Batok West Ave 2,POINT (17811.742 38223.842)


In [39]:
bus_loc['PLN_AREA_N'] = bus_loc.geometry.apply(get_PLA_AREA_N)

bus_loc.head()

,RequestTimestamp,BusStopCode,RoadName,Description,geometry,PLN_AREA_N
0,2019.02.01.03.00.00,43759,Bt Batok Rd,Blk 443D,POINT (17269.854 37561.743),BUKIT BATOK
1,2019.02.01.03.00.00,43761,Bt Batok Rd,Opp Dulwich Coll,POINT (17354.077 36989.704),TENGAH
2,2019.02.01.03.00.00,43769,Bt Batok Rd,Dulwich Coll/Blk 445,POINT (17371.596 37069.306),BUKIT BATOK
3,2019.02.01.03.00.00,43771,Bt Batok Rd,Bef Bt Batok West Ave 2,POINT (17766.452 38229.766),TENGAH
4,2019.02.01.03.00.00,43779,Bt Batok Rd,Aft Bt Batok West Ave 2,POINT (17811.742 38223.842),BUKIT BATOK


In [40]:
# transforn the postcode colume into the same datatype
bus_vol.ORIGIN_PT_CODE = bus_vol.ORIGIN_PT_CODE.astype('int64')
bus_vol.DESTINATION_PT_CODE = bus_vol.DESTINATION_PT_CODE.astype('int64')

In [41]:
# check the datatype
bus_vol.dtypes

YEAR_MONTH             object
DAY_TYPE               object
TIME_PER_HOUR           int64
PT_TYPE                object
ORIGIN_PT_CODE          int64
DESTINATION_PT_CODE     int64
TOTAL_TRIPS             int64
dtype: object

In [42]:
bus_loc.dtypes

RequestTimestamp      object
BusStopCode            int64
RoadName              object
Description           object
geometry            geometry
PLN_AREA_N            object
dtype: object

In [43]:
bus_loc.head()

,RequestTimestamp,BusStopCode,RoadName,Description,geometry,PLN_AREA_N
0,2019.02.01.03.00.00,43759,Bt Batok Rd,Blk 443D,POINT (17269.854 37561.743),BUKIT BATOK
1,2019.02.01.03.00.00,43761,Bt Batok Rd,Opp Dulwich Coll,POINT (17354.077 36989.704),TENGAH
2,2019.02.01.03.00.00,43769,Bt Batok Rd,Dulwich Coll/Blk 445,POINT (17371.596 37069.306),BUKIT BATOK
3,2019.02.01.03.00.00,43771,Bt Batok Rd,Bef Bt Batok West Ave 2,POINT (17766.452 38229.766),TENGAH
4,2019.02.01.03.00.00,43779,Bt Batok Rd,Aft Bt Batok West Ave 2,POINT (17811.742 38223.842),BUKIT BATOK


In [44]:
bus_vol1 = bus_loc.set_index(['BusStopCode'])['PLN_AREA_N'].to_dict()
bus_vol1

def bus_stop_loc(PT_CODE):
    return bus_vol1.get(PT_CODE)

bus_vol['ORIGIN_loc'] = bus_vol.ORIGIN_PT_CODE.apply(bus_stop_loc)

bus_vol['DESTINATION_loc'] = bus_vol.DESTINATION_PT_CODE.apply(bus_stop_loc)
bus_vol.head()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS,ORIGIN_loc,DESTINATION_loc
0,2019-01,WEEKDAY,6,BUS,95209,93019,34,CHANGI,BEDOK
1,2019-01,WEEKENDS/HOLIDAY,15,BUS,10149,50051,1,BUKIT MERAH,NOVENA
2,2019-01,WEEKDAY,15,BUS,10149,50051,2,BUKIT MERAH,NOVENA
3,2019-01,WEEKDAY,23,BUS,64319,63139,1,HOUGANG,HOUGANG
4,2019-01,WEEKDAY,8,BUS,85049,92149,1,BEDOK,BEDOK


Now, I have linked all the bus volume data with its origin and desitination, then its time to deal with the train data. 

## Train flux

In [5]:
# read the MRT transpotation volume data 
train_vol = pd.read_csv(path + '\TrainODvolume.2019.01.csv')
train_vol.head()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,2019-01,WEEKDAY,9,TRAIN,DT24,EW32,1
1,2019-01,WEEKDAY,9,TRAIN,EW32,DT24,1
2,2019-01,WEEKENDS/HOLIDAY,9,TRAIN,EW32,DT24,2
3,2019-01,WEEKDAY,6,TRAIN,BP4,EW31,2
4,2019-01,WEEKDAY,7,TRAIN,DT26,CC26,9


In [6]:
train_loc = gpd.read_file(path + '\mrtsg.csv')
train_loc['geometry']=gpd.points_from_xy(train_loc['X'],train_loc['Y'])
train_loc['PLN_AREA_N'] = train_loc.geometry.apply(get_PLA_AREA_N)

In [7]:
train_loc

,OBJECTID,STN_NAME,STN_NO,X,Y,Latitude,Longitude,COLOR,geometry,PLN_AREA_N
0,12,ADMIRALTY MRT STATION,NS10,24402.1063,46918.1131,1.440585001,103.8009982,RED,POINT (24402.106 46918.113),WOODLANDS
1,16,ALJUNIED MRT STATION,EW9,33518.6049,33190.0020,1.316432612,103.882893,GREEN,POINT (33518.605 33190.002),GEYLANG
2,33,ANG MO KIO MRT STATION,NS16,29807.2655,39105.7720,1.369933175,103.8495535,RED,POINT (29807.266 39105.772),ANG MO KIO
3,81,BAKAU LRT STATION,SE3,36026.0821,41113.8766,1.388092704,103.9054179,OTHERS,POINT (36026.082 41113.877),SENGKANG
4,80,BANGKIT LRT STATION,BP9,21248.2460,40220.9693,1.380017897,103.7726667,OTHERS,POINT (21248.246 40220.969),BUKIT PANJANG
...,...,...,...,...,...,...,...,...,...,...
184,175,WOODLANDS SOUTH MRT STATION,TE3,23607.8309,45444.7113,1.427259979,103.7938631,OTHERS,POINT (23607.831 45444.711),WOODLANDS
185,146,WOODLEIGH MRT STATION,NE11,32173.3186,35706.3794,1.339190046,103.8708081,PURPLE,POINT (32173.319 35706.379),TOA PAYOH
186,6,YEW TEE MRT STATION,NS5,18438.9791,42158.0124,1.397535018,103.7474307,RED,POINT (18438.979 42158.012),CHOA CHU KANG
187,41,YIO CHU KANG MRT STATION,NS15,29294.1283,40413.0820,1.381756046,103.8449439,RED,POINT (29294.128 40413.082),ANG MO KIO


Find what station names are 'nicknames': 

In [11]:
def newname():
    a = pd.concat([train_loc['STN_NO'],train_vol['ORIGIN_PT_CODE'],train_vol['ORIGIN_PT_CODE']])
    result = pd.DataFrame()
    result['STN_NO'] = a.drop_duplicates(keep=False)
    result=result.reset_index(drop=True)
    #result only contains the stations that's full No. is in loc.file but the vol.file
    
    def split(string):
        string = str(string)
        try:
            string.split('/')[1]
        except IndexError: 
            return [string],[None],[None]
        else:
            try:
                string.split('/')[2]
            except IndexError: 
                return string.split('/')[0].split(), string.split('/')[1].split() ,[None]
            else:        
                return string.split('/')[0].split(), string.split('/')[1].split(), string.split('/')[2].split()
    #This function is used to seperate the 'nicknames' of a station
    
    result['namelist']=result['STN_NO'].apply(split)
    
    def get_name1(namelist):
        return namelist[0][0]
    def get_name2(namelist):
        return namelist[1][0]
    def get_name3(namelist):
        return namelist[2][0]

    result['name1']=result['namelist'].apply(get_name1)
    result['name2']=result['namelist'].apply(get_name2)
    result['name3']=result['namelist'].apply(get_name3)

    result = result.drop(columns=['namelist'])
    
    list1=result.set_index(['name1'])['STN_NO'].to_dict() 
    list2=result.set_index(['name2'])['STN_NO'].to_dict() 
    list3=result.set_index(['name3'])['STN_NO'].to_dict()
    
    def get_name(name): 
        if name in list1.keys():
            get_name =list1.get(name)
        elif oldname in list2.keys():
            get_name =list2.get(name)
        elif oldname in list3.keys():
            get_name =list3.get(name)
        else: get_name=oldname 
        return get_name
      
    return list1,list2,list3

Use a easier function 

In [12]:
list1,list2,list3 = newname()

def get_name(oldname): 
    if oldname in list1.keys():
        get_name=list1.get(oldname)
    elif oldname in list2.keys():
        get_name =list2.get(oldname)
    elif oldname in list3.keys():
        get_name =list3.get(oldname)
    else: get_name=oldname 
    return get_name

test it

In [13]:
train_vol['ORIGIN_PT_CODE']=train_vol['ORIGIN_PT_CODE'].apply(get_name)

In [14]:
train_vol['DESTINATION_PT_CODE'] = train_vol['DESTINATION_PT_CODE'].apply(get_name)

In [45]:
train_vol.head()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS,ORIGIN_loc,DESTINATION_loc
0,2019-01,WEEKDAY,9,TRAIN,DT24,EW32,1,KALLANG,TUAS
1,2019-01,WEEKDAY,9,TRAIN,EW32,DT24,1,TUAS,KALLANG
2,2019-01,WEEKENDS/HOLIDAY,9,TRAIN,EW32,DT24,2,TUAS,KALLANG
3,2019-01,WEEKDAY,6,TRAIN,BP4,EW31,2,CHOA CHU KANG,TUAS
4,2019-01,WEEKDAY,7,TRAIN,DT26,CC26,9,GEYLANG,QUEENSTOWN


We can see all the short names are replaced by the full names. 

In [47]:
train_loc.head()

,OBJECTID,STN_NAME,STN_NO,X,Y,Latitude,Longitude,COLOR,geometry,PLN_AREA_N
0,12,ADMIRALTY MRT STATION,NS10,24402.1063,46918.1131,1.440585001,103.8009982,RED,POINT (24402.106 46918.113),WOODLANDS
1,16,ALJUNIED MRT STATION,EW9,33518.6049,33190.0020,1.316432612,103.882893,GREEN,POINT (33518.605 33190.002),GEYLANG
2,33,ANG MO KIO MRT STATION,NS16,29807.2655,39105.7720,1.369933175,103.8495535,RED,POINT (29807.266 39105.772),ANG MO KIO
3,81,BAKAU LRT STATION,SE3,36026.0821,41113.8766,1.388092704,103.9054179,OTHERS,POINT (36026.082 41113.877),SENGKANG
4,80,BANGKIT LRT STATION,BP9,21248.2460,40220.9693,1.380017897,103.7726667,OTHERS,POINT (21248.246 40220.969),BUKIT PANJANG


In [26]:
train_loc_dict = train_loc.set_index(['STN_NO'])['PLN_AREA_N'].to_dict()

def find_train_loc(PT_CODE):
    return train_loc_dict.get(PT_CODE)

train_vol['ORIGIN_loc'] = train_vol['ORIGIN_PT_CODE'].apply(find_train_loc)
train_vol['DESTINATION_loc'] = train_vol['DESTINATION_PT_CODE'].apply(find_train_loc)

In [27]:
train_vol

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS,ORIGIN_loc,DESTINATION_loc
0,2019-01,WEEKDAY,9,TRAIN,DT24,EW32,1,KALLANG,TUAS
1,2019-01,WEEKDAY,9,TRAIN,EW32,DT24,1,TUAS,KALLANG
2,2019-01,WEEKENDS/HOLIDAY,9,TRAIN,EW32,DT24,2,TUAS,KALLANG
3,2019-01,WEEKDAY,6,TRAIN,BP4,EW31,2,CHOA CHU KANG,TUAS
4,2019-01,WEEKDAY,7,TRAIN,DT26,CC26,9,GEYLANG,QUEENSTOWN
...,...,...,...,...,...,...,...,...,...
801139,2019-01,WEEKENDS/HOLIDAY,5,TRAIN,BP2,BP7,14,CHOA CHU KANG,BUKIT PANJANG
801140,2019-01,WEEKDAY,21,TRAIN,CC29,NE13,898,BUKIT MERAH,HOUGANG
801141,2019-01,WEEKDAY,21,TRAIN,NE13,CC29,105,HOUGANG,BUKIT MERAH
801142,2019-01,WEEKENDS/HOLIDAY,21,TRAIN,CC29,NE13,455,BUKIT MERAH,HOUGANG


## Sum up the flux

In [48]:
flux_bus_total = pd.DataFrame()

flux_bus_total = bus_vol.groupby('ORIGIN_loc').sum()['TOTAL_TRIPS'].reset_index()
flux_bus_total.to_csv(path+'\Total_bus_flux2019.csv')

In [49]:
flux_bus_total

,ORIGIN_loc,TOTAL_TRIPS
0,ANG MO KIO,5753076
1,BEDOK,8076632
2,BISHAN,2680822
3,BOON LAY,186710
4,BUKIT BATOK,3588419
5,BUKIT MERAH,4666840
6,BUKIT PANJANG,2520233
7,BUKIT TIMAH,2222822
8,CENTRAL WATER CATCHMENT,168233
9,CHANGI,1164105


In [50]:
flux_bus_ij = pd.DataFrame()

flux_bus_ij = bus_vol.groupby(['ORIGIN_loc','DESTINATION_loc']).sum()['TOTAL_TRIPS'].reset_index()

flux_bus_ij.to_csv(path+'\Total_bus_flux_ij2019.csv')

In [51]:
flux_bus_ij

,ORIGIN_loc,DESTINATION_loc,TOTAL_TRIPS
0,ANG MO KIO,ANG MO KIO,3438644
1,ANG MO KIO,BEDOK,60610
2,ANG MO KIO,BISHAN,505646
3,ANG MO KIO,BUKIT BATOK,3501
4,ANG MO KIO,BUKIT MERAH,13361
...,...,...,...
1122,YISHUN,TAMPINES,101165
1123,YISHUN,TANGLIN,1563
1124,YISHUN,TOA PAYOH,23422
1125,YISHUN,WOODLANDS,196778


In [34]:
flux_MRT_total = pd.DataFrame()

flux_MRT_total = train_vol.groupby('ORIGIN_loc').sum()['TOTAL_TRIPS'].reset_index()
flux_MRT_total.to_csv(path+'\Total_MRT_flux2019.csv')

In [35]:
flux_MRT_total

,ORIGIN_loc,TOTAL_TRIPS
0,ANG MO KIO,2537780
1,BEDOK,2822888
2,BISHAN,1861313
3,BUKIT BATOK,2393861
4,BUKIT MERAH,4074769
5,BUKIT PANJANG,1522764
6,BUKIT TIMAH,1451151
7,CHANGI,593403
8,CHOA CHU KANG,2720252
9,CLEMENTI,1432139


In [36]:
flux_MRT_ij = pd.DataFrame()

flux_MRT_ij = train_vol.groupby(['ORIGIN_loc','DESTINATION_loc']).sum()['TOTAL_TRIPS'].reset_index()

flux_MRT_ij.to_csv(path+'\Total_MRT_flux_ij2019.csv')